In [9]:
import requests
import datetime

# Define function to get current price of Bitcoin
def get_current_price():
    url = "https://www.deribit.com/api/v2/public/ticker?instrument_name=BTC-PERPETUAL"
    response = requests.get(url)
    data = response.json()
    return data['result']['index_price']

# Fetching option prices from Deribit for Bitcoin
def fetch_deribit_option_data():
    url = "https://www.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=option&expired=false"
    response = requests.get(url)
    data = response.json()
    return data['result']

# Filter options to get near the money options
def filter_near_the_money_options(options, current_price, option_type, time_periods):
    near_the_money_options = {}
    for period_name, period_duration in time_periods.items():
        near_the_money_options[period_name] = []
        current_time = int(datetime.datetime.now().timestamp() * 1000)
        for option in options:
            if option['option_type'] == option_type and current_time + period_duration >= option['expiration_timestamp'] >= current_time:
                if abs(option['strike'] - current_price) < current_price * 0.01:  # within 1% of the current price
                    near_the_money_options[period_name].append(option)
    return near_the_money_options

# Fetch option market data (bid and ask prices)
def fetch_option_market_data(option_name):
    url = f"https://www.deribit.com/api/v2/public/ticker?instrument_name={option_name}"
    response = requests.get(url)
    data = response.json()
    return data['result']

# Extract relevant information from options
def extract_relevant_info(options, market_data, option_type):
    filtered_info = []
    for period, opts in options.items():
        for opt in opts:
            market = market_data.get(opt['instrument_name'], {})
            filtered_info.append({
                'Option Type': option_type,
                'Strike Price': opt['strike'],
                'Expiration Date': datetime.datetime.fromtimestamp(opt['expiration_timestamp'] / 1000),
                'Bid Price': market.get('best_bid_price'),
                'Ask Price': market.get('best_ask_price')
            })
    return filtered_info

# Main process
current_price = get_current_price()
options = fetch_deribit_option_data()

# Define time periods in milliseconds (1 day, 1 month, 1 quarter, 4 quarters)
time_periods = {
    "1 day": 86400000,
    "1 month": 2592000000,
    "1 quarter": 7776000000,
    "4 quarters": 31536000000
}

# Collect near the money options
near_the_money_calls = filter_near_the_money_options(options, current_price, "call", time_periods)
near_the_money_puts = filter_near_the_money_options(options, current_price, "put", time_periods)

# Collect market data for each near the money option
market_data = {}
for period, calls in near_the_money_calls.items():
    for call in calls:
        market_data[call['instrument_name']] = fetch_option_market_data(call['instrument_name'])

for period, puts in near_the_money_puts.items():
    for put in puts:
        market_data[put['instrument_name']] = fetch_option_market_data(put['instrument_name'])

# Extract relevant information
calls_info = extract_relevant_info(near_the_money_calls, market_data, "call")
puts_info = extract_relevant_info(near_the_money_puts, market_data, "put")

# Combine calls and puts info
combined_info = calls_info + puts_info

# Output the filtered information
import pandas as pd
df = pd.DataFrame(combined_info)
df

,Option Type,Strike Price,Expiration Date,Bid Price,Ask Price
0,call,66000.0,2024-06-17 09:00:00,0.0075,0.0130
1,call,66500.0,2024-06-17 09:00:00,0.0046,0.0050
2,call,67000.0,2024-06-17 09:00:00,0.0018,0.0022
3,call,66000.0,2024-06-17 09:00:00,0.0075,0.0130
4,call,66500.0,2024-06-17 09:00:00,0.0046,0.0050
...,...,...,...,...,...
113,put,67000.0,2024-07-05 09:00:00,0.0400,0.0410
114,put,66000.0,2024-07-26 09:00:00,0.0520,0.0535
115,put,67000.0,2024-07-26 09:00:00,0.0595,0.0610
116,put,66000.0,2024-08-30 09:00:00,0.0765,0.0780


In [3]:
import requests
import datetime
import pandas as pd

# Define function to get current price of Bitcoin
def get_current_price():
    url = "https://www.deribit.com/api/v2/public/ticker?instrument_name=BTC-PERPETUAL"
    response = requests.get(url)
    data = response.json()
    return data['result']['index_price']

# Fetching option prices from Deribit for Bitcoin
def fetch_deribit_option_data():
    url = "https://www.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=option&expired=false"
    response = requests.get(url)
    data = response.json()
    return data['result']

# Filter options to get near the money options
def filter_near_the_money_options(options, current_price, option_type, time_periods):
    near_the_money_options = {}
    for period_name, period_duration in time_periods.items():
        near_the_money_options[period_name] = []
        current_time = int(datetime.datetime.now().timestamp() * 1000)
        for option in options:
            if option['option_type'] == option_type and current_time + period_duration >= option['expiration_timestamp'] >= current_time:
                if abs(option['strike'] - current_price) < current_price * 0.01:  # within 1% of the current price
                    near_the_money_options[period_name].append(option)
    return near_the_money_options

# Fetch option market data (bid and ask prices)
def fetch_option_market_data(option_name):
    url = f"https://www.deribit.com/api/v2/public/ticker?instrument_name={option_name}"
    response = requests.get(url)
    data = response.json()
    return data['result']

# Extract relevant information from options
def extract_relevant_info(options, market_data, option_type, underlying_price):
    filtered_info = []
    for period, opts in options.items():
        for opt in opts:
            market = market_data.get(opt['instrument_name'], {})
            filtered_info.append({
                'Option Type': option_type,
                'Strike Price': opt['strike'],
                'Expiration Date': datetime.datetime.fromtimestamp(opt['expiration_timestamp'] / 1000),
                'Bid Price': market.get('best_bid_price'),
                'Ask Price': market.get('best_ask_price'),
                'Underlying Price': underlying_price,
            })
    return filtered_info

# Main process
current_price = get_current_price()
options = fetch_deribit_option_data()

# Define time periods in milliseconds (1 day, 1 month, 1 quarter, 4 quarters)
time_periods = {
    "1 day": 86400000,
    "1 month": 2592000000,
    "1 quarter": 7776000000,
    "4 quarters": 31536000000
}

# Collect near the money options
near_the_money_calls = filter_near_the_money_options(options, current_price, "call", time_periods)
near_the_money_puts = filter_near_the_money_options(options, current_price, "put", time_periods)

# Collect market data for each near the money option
market_data = {}
for period, calls in near_the_money_calls.items():
    for call in calls:
        market_data[call['instrument_name']] = fetch_option_market_data(call['instrument_name'])

for period, puts in near_the_money_puts.items():
    for put in puts:
        market_data[put['instrument_name']] = fetch_option_market_data(put['instrument_name'])

# Extract relevant information
calls_info = extract_relevant_info(near_the_money_calls, market_data, "call", current_price)
puts_info = extract_relevant_info(near_the_money_puts, market_data, "put", current_price)

# Combine calls and puts info
combined_info = calls_info + puts_info

# Output the filtered information
df = pd.DataFrame(combined_info)
df




,Option Type,Strike Price,Expiration Date,Bid Price,Ask Price,Underlying Price
0,call,66000.0,2024-06-17 09:00:00,0.0070,0.0130,66570.13
1,call,66500.0,2024-06-17 09:00:00,0.0048,0.0055,66570.13
2,call,67000.0,2024-06-17 09:00:00,0.0019,0.0020,66570.13
3,call,66000.0,2024-06-17 09:00:00,0.0070,0.0130,66570.13
4,call,66500.0,2024-06-17 09:00:00,0.0048,0.0055,66570.13
...,...,...,...,...,...,...
113,put,67000.0,2024-07-05 09:00:00,0.0395,0.0405,66570.13
114,put,66000.0,2024-07-26 09:00:00,0.0520,0.0530,66570.13
115,put,67000.0,2024-07-26 09:00:00,0.0595,0.0605,66570.13
116,put,66000.0,2024-08-30 09:00:00,0.0760,0.0775,66570.13


In [6]:
import requests
import datetime
import pandas as pd

# Define function to get current price of Bitcoin
def get_current_price():
    url = "https://www.deribit.com/api/v2/public/ticker?instrument_name=BTC-PERPETUAL"
    response = requests.get(url)
    data = response.json()
    return data['result']['index_price']

# Fetching option prices from Deribit for Bitcoin
def fetch_deribit_option_data():
    url = "https://www.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=option&expired=false"
    response = requests.get(url)
    data = response.json()
    return data['result']

# Filter options to get near the money options
def filter_near_the_money_options(options, current_price, option_type, time_periods):
    near_the_money_options = {}
    for period_name, period_duration in time_periods.items():
        near_the_money_options[period_name] = []
        current_time = int(datetime.datetime.now().timestamp() * 1000)
        for option in options:
            if option['option_type'] == option_type and current_time + period_duration >= option['expiration_timestamp'] >= current_time:
                if abs(option['strike'] - current_price) < current_price * 0.01:  # within 1% of the current price
                    near_the_money_options[period_name].append(option)
    return near_the_money_options

# Fetch option market data (bid and ask prices)
def fetch_option_market_data(option_name):
    url = f"https://www.deribit.com/api/v2/public/ticker?instrument_name={option_name}"
    response = requests.get(url)
    data = response.json()
    return data['result']

# Extract relevant information from options
def extract_relevant_info(options, market_data, option_type, underlying_price):
    filtered_info = []
    for period, opts in options.items():
        for opt in opts:
            market = market_data.get(opt['instrument_name'], {})
            filtered_info.append({
                'Option Type': option_type,
                'Strike Price': opt['strike'],
                'Expiration Date': datetime.datetime.fromtimestamp(opt['expiration_timestamp'] / 1000),
                'Bid Price': market.get('best_bid_price'),
                'Ask Price': market.get('best_ask_price'),
                'Underlying Price': underlying_price,
                'Estimated Delivery Price': market.get('estimated_delivery_price')
            })
    return filtered_info

# Main process
current_price = get_current_price()
options = fetch_deribit_option_data()

# Define time periods in milliseconds (1 day, 1 month, 1 quarter, 4 quarters)
time_periods = {
    "1 day": 86400000,
    "1 month": 2592000000,
    "1 quarter": 7776000000,
    "4 quarters": 31536000000
}

# Collect near the money options
near_the_money_calls = filter_near_the_money_options(options, current_price, "call", time_periods)
near_the_money_puts = filter_near_the_money_options(options, current_price, "put", time_periods)

# Collect market data for each near the money option
market_data = {}
for period, calls in near_the_money_calls.items():
    for call in calls:
        market_data[call['instrument_name']] = fetch_option_market_data(call['instrument_name'])

for period, puts in near_the_money_puts.items():
    for put in puts:
        market_data[put['instrument_name']] = fetch_option_market_data(put['instrument_name'])

# Extract relevant information
calls_info = extract_relevant_info(near_the_money_calls, market_data, "call", current_price)
puts_info = extract_relevant_info(near_the_money_puts, market_data, "put", current_price)

# Combine calls and puts info
combined_info = calls_info + puts_info

# Output the filtered information
df = pd.DataFrame(combined_info)
df

,Option Type,Strike Price,Expiration Date,Bid Price,Ask Price,Underlying Price,Estimated Delivery Price
0,call,66000.0,2024-06-17 09:00:00,0.0065,0.0130,66541.18,66538.85
1,call,66500.0,2024-06-17 09:00:00,0.0045,0.0050,66541.18,66538.85
2,call,67000.0,2024-06-17 09:00:00,0.0017,0.0021,66541.18,66538.85
3,call,66000.0,2024-06-17 09:00:00,0.0065,0.0130,66541.18,66538.85
4,call,66500.0,2024-06-17 09:00:00,0.0045,0.0050,66541.18,66538.85
...,...,...,...,...,...,...,...
113,put,67000.0,2024-07-05 09:00:00,0.0395,0.0405,66541.18,66544.28
114,put,66000.0,2024-07-26 09:00:00,0.0520,0.0530,66541.18,66544.28
115,put,67000.0,2024-07-26 09:00:00,0.0595,0.0605,66541.18,66544.28
116,put,66000.0,2024-08-30 09:00:00,0.0765,0.0775,66541.18,66544.63
